### Import Elliott Wave Models

In [1]:
from models.WavePattern import WavePattern
from models.WaveRules import Impulse, LeadingDiagonal, Correction
from models.WaveAnalyzer import WaveAnalyzer
from models.WaveCycle import WaveCycle
from models.WaveOptions import WaveOptionsGenerator5, WaveOptionsGenerator3
from models.helpers import plot_pattern
from models.helpers import plot_cycle
from models.helpers import plot_pattern_two

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [3]:
global df
df_data = pd.read_csv(r'./data/BTC-USD.csv') # 5 years
df = df_data.iloc[len(list(df_data.Date)) - 365 :, :] # 1 year plotting
idx_start = np.argmin(np.array(list(df['Low'])))


In [4]:
plotting = go.Ohlc(x=df['Date'],
                   open=df['Open'],
                   high=df['High'],
                   low=df['Low'],
                   close=df['Close'])

layout = dict(title="Stocks Plotting")
fig = go.Figure(data=[plotting], layout=layout)
fig.update(layout_xaxis_rangeslider_visible=False)

fig.show()

In [5]:
wa = WaveAnalyzer(df=df, verbose=False)
wave_options_impulse = WaveOptionsGenerator5(up_to=15)  # generates WaveOptions up to [15, 15, 15, 15, 15]
wave_options_correction = WaveOptionsGenerator3(up_to=9) 

In [6]:
impulse = Impulse('impulse')
leading_diagonal = LeadingDiagonal('leading diagonal')
correction = Correction('correction')
rules_to_check = [impulse, correction]

In [7]:
print(f'Start at idx: {idx_start}')
print(f"will run up to {wave_options_impulse.number / 1e6}M combinations.")

Start at idx: 78
will run up to 0.579195M combinations.


In [8]:
wavepatterns_up = list()
wavepatterns_down = list()
completeList = list()

In [9]:

try:
    for new_option_impulse in wave_options_impulse.options_sorted:

        waves_up = wa.find_impulsive_wave(idx_start=idx_start, wave_config=new_option_impulse.values)

        if waves_up:
            wavepattern_up = WavePattern(waves_up, verbose=True)

            for rule in rules_to_check:

                if wavepattern_up.check_rule(rule):
                    if wavepattern_up in wavepatterns_up:
                        continue
                    else:
                        wavepatterns_up.append(wavepattern_up)
                        print(f'{rule.name} found: {new_option_impulse.values}')
                        cor_end = waves_up[4].idx_end
                        cor_date = waves_up[4].date_end
                        cor_close = waves_up[4].high
                        cor_low = waves_up[4].low
except Exception as e:
    print("\n\n\n",e)
    

print(cor_end) 

Rule Violation of impulse for condition w4_1: End of Wave4 is lower than End of Wave1
Rule Violation of correction for condition w2_1: End of WaveB is higher than Start of WaveA.
Rule Violation of impulse for condition w3_2: End of Wave3 is lower than End of Wave1
Rule Violation of correction for condition w2_1: End of WaveB is higher than Start of WaveA.
Rule Violation of impulse for condition w3_1: Wave3 is the shortest Wave.
Rule Violation of correction for condition w2_1: End of WaveB is higher than Start of WaveA.
Rule Violation of impulse for condition w4_1: End of Wave4 is lower than End of Wave1
Rule Violation of correction for condition w2_1: End of WaveB is higher than Start of WaveA.
impulse found: [1, 3, 2, 0, 0]
Rule Violation of correction for condition w2_1: End of WaveB is higher than Start of WaveA.
impulse found: [1, 3, 3, 0, 0]
Rule Violation of correction for condition w2_1: End of WaveB is higher than Start of WaveA.
impulse found: [1, 3, 4, 0, 0]
Rule Violation of

In [10]:
end_date = str(cor_date)
imp_end_date_idx = int(np.where(df_data['Date'] == end_date)[0])
imp_end_date_idx

1715

In [11]:
# Corrective Pattern points
try:
    wave_cycles = set()
    for new_option_correction in wave_options_correction.options_sorted:
        waves_cor = wa.find_corrective_wave(idx_start=cor_end, wave_config=new_option_correction.values)

        if waves_cor:
            wavepattern_cor = WavePattern(waves_cor, verbose=True)

            for rule in rules_to_check:

                if wavepattern_cor.check_rule(rule):
                    if wavepattern_cor in wavepatterns_down:
                        continue
                    else:
                        wavepatterns_down.append(wavepattern_cor)
                        print(f'{rule.name} found: {new_option_correction.values}')

except Exception as e:
    print("\n\n\n", e)

Rule Violation of impulse for condition w2_1: End of Wave2 is lower than Start of Wave1.
correction found: [0, 0, 0, None, None]
Rule Violation of impulse for condition w2_1: End of Wave2 is lower than Start of Wave1.
Rule Violation of correction for condition w2_5: WaveC shorter (value wise) than 0.60 x WaveA.
Rule Violation of impulse for condition w2_1: End of Wave2 is lower than Start of Wave1.
Rule Violation of correction for condition w2_5: WaveC shorter (value wise) than 0.60 x WaveA.
Rule Violation of impulse for condition w2_1: End of Wave2 is lower than Start of Wave1.
correction found: [3, 0, 0, None, None]
Rule Violation of impulse for condition w2_1: End of Wave2 is lower than Start of Wave1.
Rule Violation of correction for condition w2_2: End of WaveB is higher than Start of WaveA.
Rule Violation of impulse for condition w2_1: End of Wave2 is lower than Start of Wave1.
Rule Violation of correction for condition w2_2: End of WaveB is higher than Start of WaveA.
Rule Viola

In [12]:
# plot_pattern(df=df, wave_pattern=wavepatterns_down, title="Corrective Wave")
wavepatterns_down

In [13]:
import os
try:
    if(wavepatterns_up != [] or wavepatterns_down != []):
        completeWave=WaveCycle(wavepatterns_up[-1],wavepatterns_down[-1])
        plot_pattern(df=df, wave_pattern=wavepatterns_up[-1], title="Impulsive Wave")
        plot_pattern(df=df, wave_pattern=wavepatterns_down[-1], title="Corrective Wave")
        plot_cycle(df=df, wave_cycle=completeWave, title="Elliot Wave Plotting")
except:
    if(wavepatterns_up == []):
        print("Cannot Plot Elliott Wave due to insufficient trends in impulsive wave")
    elif(wavepatterns_down == []):
        print("Cannot Plot Elliott Wave due to insufficient trends in corrective wave")
    # os._exit(0)



### Deep Learning Model

In [14]:
dataset_train_open = df_data.iloc[:, 1:2]
dataset_train_high = df_data.iloc[:, 2:3]
dataset_train_low = df_data.iloc[:, 3:4]
dataset_train_close = df_data.iloc[:, 4:5]

In [15]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))

dataset_train_open_scaled = sc.fit_transform(dataset_train_open)
dataset_train_high_scaled = sc.fit_transform(dataset_train_high)
dataset_train_low_scaled = sc.fit_transform(dataset_train_low)
dataset_train_close_scaled = sc.fit_transform(dataset_train_close)

In [16]:
# Open
X_train_open = []
y_train_open = []
for i in range(60, len(dataset_train_open_scaled)):
    X_train_open.append(dataset_train_open_scaled[i-60:i, 0])
    y_train_open.append(dataset_train_open_scaled[i, 0])
X_train_open, y_train_open = np.array(X_train_open), np.array(y_train_open)
X_train_open

array([[0.00365543, 0.00432557, 0.00471633, ..., 0.00518426, 0.00572259,
        0.00572409],
       [0.00432557, 0.00471633, 0.00511746, ..., 0.00572259, 0.00572409,
        0.00578459],
       [0.00471633, 0.00511746, 0.00477008, ..., 0.00572409, 0.00578459,
        0.00618136],
       ...,
       [0.56363046, 0.56756286, 0.54055754, ..., 0.69836552, 0.69243952,
        0.6698016 ],
       [0.56756286, 0.54055754, 0.54362692, ..., 0.69243952, 0.6698016 ,
        0.68078044],
       [0.54055754, 0.54362692, 0.60896255, ..., 0.6698016 , 0.68078044,
        0.67437974]])

In [17]:
#High
X_train_high = []
y_train_high = []
for i in range(60, len(dataset_train_high_scaled)):
    X_train_high.append(dataset_train_high_scaled[i-60:i, 0])
    y_train_high.append(dataset_train_high_scaled[i, 0])
X_train_high, y_train_high = np.array(X_train_high), np.array(y_train_high)

#        <--- axis 1 --->   axis 2          axis 1 -> rows 
# X_train 60th - ith date in col 0          axis 2 -> cols
# y_train is ith date

In [18]:
#Low
X_train_low = []
y_train_low = []
for i in range(60, len(dataset_train_low_scaled)):
    X_train_low.append(dataset_train_low_scaled[i-60:i, 0])
    y_train_low.append(dataset_train_low_scaled[i, 0])
X_train_low, y_train_low = np.array(X_train_low), np.array(y_train_low)

#        <--- axis 1 --->   axis 2          axis 1 -> rows 
# X_train 60th - ith date in col 0          axis 2 -> cols
# y_train is ith date

In [19]:
# Close
X_train_close = []
y_train_close = []
for i in range(60, len(dataset_train_close_scaled)):
    X_train_close.append(dataset_train_close_scaled[i-60:i, 0])
    y_train_close.append(dataset_train_close_scaled[i, 0])
X_train_close, y_train_close = np.array(X_train_close), np.array(y_train_close)

In [20]:
"""
X_train reshape process
requires 3d data for lstm
1) batch_size -> np.shape[0]
2) timesteps or lookback -> np.shape[1]
3) input dimension -> 1 because time series data so 1d data 

This reshape converts 2d -> 3d 
"""
X_train_open = np.reshape(X_train_open, (X_train_open.shape[0], X_train_open.shape[1], 1))
X_train_high = np.reshape(X_train_high, (X_train_high.shape[0], X_train_high.shape[1], 1))
X_train_low = np.reshape(X_train_low, (X_train_low.shape[0], X_train_low.shape[1], 1))
X_train_close = np.reshape(X_train_close, (X_train_close.shape[0], X_train_close.shape[1], 1))


In [21]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout, Bidirectional, LSTM
from keras.layers import Conv1D, MaxPooling1D, Flatten
from keras.models import save_model, load_model

### Import Saved Model

In [22]:
model_open = load_model('save_model/open_model')
model_high = load_model('save_model/high_model')
model_low = load_model('save_model/low_model')
model_close = load_model('save_model/close_model')
model_open.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 60, 32)            64        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 60, 32)           0         
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (None, 60, 100)          33200     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 60, 100)           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 60, 100)          60400     
 nal)                                                            
                                                        

### Open Value Model

In [23]:

# model_open = Sequential()

# # CNN

# model_open.add(Conv1D(filters=32, kernel_size=1, activation='relu', input_shape=(X_train_open.shape[1], 1)))
# model_open.add(MaxPooling1D(pool_size=1))

# # LSTM

# model_open.add(Bidirectional(LSTM(50, return_sequences=True)))
# model_open.add(Dropout(0.2)) # Dropout 20% of neurons

# model_open.add(Bidirectional(LSTM(50, return_sequences=True)))
# model_open.add(Dropout(0.1))

# model_open.add(Bidirectional(LSTM(50, return_sequences=True)))
# model_open.add(Dropout(0.1))

# model_open.add(LSTM(50))
# model_open.add(Dropout(0.2))

# model_open.add(Flatten())

# model_open.add(Dense(1))

# model_open.compile(optimizer='adam', loss='mean_squared_error')

In [24]:
# model_open.fit(X_train_high, y_train_high, epochs = 150, batch_size = 32)
# model_open.save('save_model/open_model')

### High Value Model

In [25]:

# model_high = Sequential()

# # CNN

# model_high.add(Conv1D(filters=32, kernel_size=1, activation='relu', input_shape=(X_train_high.shape[1], 1)))
# model_high.add(MaxPooling1D(pool_size=1))

# # LSTM

# model_high.add(Bidirectional(LSTM(50, return_sequences=True)))
# model_high.add(Dropout(0.2)) # Dropout 20% of neurons

# model_high.add(Bidirectional(LSTM(50, return_sequences=True)))
# model_high.add(Dropout(0.1))

# model_high.add(Bidirectional(LSTM(50, return_sequences=True)))
# model_high.add(Dropout(0.1))

# model_high.add(LSTM(50))
# model_high.add(Dropout(0.2))

# model_high.add(Flatten())

# model_high.add(Dense(1))

# model_high.compile(optimizer='adam', loss='mean_squared_error')

In [26]:
# model_high.fit(X_train_high, y_train_high, epochs = 150, batch_size = 32)
# model_high.save('save_model/high_model')

### Low Value Model

In [27]:

# model_low = Sequential()

# # CNN

# model_low.add(Conv1D(filters=32, kernel_size=1, activation='relu', input_shape=(X_train_low.shape[1], 1)))
# model_low.add(MaxPooling1D(pool_size=1))

# # LSTM

# model_low.add(Bidirectional(LSTM(50, return_sequences=True)))
# model_low.add(Dropout(0.2)) # Dropout 20% of neurons

# model_low.add(Bidirectional(LSTM(50, return_sequences=True)))
# model_low.add(Dropout(0.1))

# model_low.add(Bidirectional(LSTM(50, return_sequences=True)))
# model_low.add(Dropout(0.1))

# model_low.add(LSTM(50))
# model_low.add(Dropout(0.2))

# model_low.add(Flatten())

# model_low.add(Dense(1))

# model_low.compile(optimizer='adam', loss='mean_squared_error')

In [28]:
# # X_train is 3d data, y_train is 1d data 
# model_low.fit(X_train_low, y_train_low, epochs = 150, batch_size = 32)
# model_low.save('save_model/low_model')

### Close Model

In [29]:

# model_close = Sequential()

# # CNN

# model_close.add(Conv1D(filters=32, kernel_size=1, activation='relu', input_shape=(X_train_close.shape[1], 1)))
# model_close.add(MaxPooling1D(pool_size=1))

# # LSTM

# model_close.add(Bidirectional(LSTM(50, return_sequences=True)))
# model_close.add(Dropout(0.2)) # Dropout 20% of neurons

# model_close.add(Bidirectional(LSTM(50, return_sequences=True)))
# model_close.add(Dropout(0.1))

# model_close.add(Bidirectional(LSTM(50, return_sequences=True)))
# model_close.add(Dropout(0.1))

# model_close.add(LSTM(50))
# model_close.add(Dropout(0.2))

# model_close.add(Flatten())

# model_close.add(Dense(1))

# model_close.compile(optimizer='adam', loss='mean_squared_error')

In [30]:
# model_low.fit(X_train_low, y_train_low, epochs = 150, batch_size = 32)
# model_low.save('save_model/close_model')

In [31]:
dataset_test_open = df_data.iloc[imp_end_date_idx:, 1:2]
dataset_test_high = df_data.iloc[imp_end_date_idx:, 2:3]
dataset_test_low = df_data.iloc[imp_end_date_idx:, 3:4]
dataset_test_close = df_data.iloc[imp_end_date_idx:, 4:5]
dataset_test_open

,Open
1715,66953.335938
1716,64978.890625
1717,64863.980469
1718,64158.121094
1719,64455.371094
...,...
1856,47062.148438
1857,45554.164063
1858,46285.500000
1859,45859.128906


### Open Value Prediction

In [32]:
dataset_total_open = pd.concat((df_data['Open'], dataset_test_open['Open']), axis = 0) # horizontal concatenation 1 vertical is 0 -> cols
inputs_open = dataset_total_open[len(dataset_total_open) - len(dataset_test_open) - 60:].values
inputs_open = inputs_open.reshape(-1,1)
inputs_open = sc.transform(inputs_open)
X_test_open = []

for i in range(60, len(inputs_open)): # for 6 months
    X_test_open.append(inputs_open[i-60:i, 0])
X_test_open = np.array(X_test_open)
X_test_open = np.reshape(X_test_open, (X_test_open.shape[0], X_test_open.shape[1], 1))
predicted_stock_price_open = model_high.predict(X_test_open)
predicted_stock_price_open = sc.inverse_transform(predicted_stock_price_open)
predicted_stock_price_open

C:\Users\91914\AppData\Local\Temp\ipykernel_22552\1241990845.py:2: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.

c:\Users\91914\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



5/5 [==============================] - 8s 40ms/step


array([[44965.477],
       [62427.85 ],
       [60904.71 ],
       [60886.953],
       [61237.902],
       [61063.062],
       [61723.   ],
       [60717.254],
       [58303.99 ],
       [58351.02 ],
       [55326.953],
       [55932.215],
       [57299.52 ],
       [56444.527],
       [54559.22 ],
       [55593.195],
       [55247.84 ],
       [56677.39 ],
       [52493.44 ],
       [53045.805],
       [55329.062],
       [55701.277],
       [55097.895],
       [55494.02 ],
       [54867.758],
       [52292.13 ],
       [47950.113],
       [47519.26 ],
       [48508.414],
       [48514.34 ],
       [48415.457],
       [45918.305],
       [45310.22 ],
       [47445.156],
       [48425.477],
       [45415.215],
       [46610.51 ],
       [47378.832],
       [46099.914],
       [44789.26 ],
       [45317.047],
       [45219.168],
       [45280.867],
       [47270.938],
       [47185.492],
       [49142.4  ],
       [49452.39 ],
       [49021.348],
       [49387.17 ],
       [49316.637],


### High Value Prediction

In [33]:
dataset_total_high = pd.concat((df_data['High'], dataset_test_high['High']), axis = 0) # horizontal concatenation 1 vertical is 0 -> cols
inputs_high = dataset_total_high[len(dataset_total_high) - len(dataset_test_high) - 60:].values
inputs_high = inputs_high.reshape(-1,1)
inputs_high = sc.transform(inputs_high)
X_test_high = []

for i in range(60, len(inputs_high)): # for 6 months
    X_test_high.append(inputs_high[i-60:i, 0])
X_test_high = np.array(X_test_high)
X_test_high = np.reshape(X_test_high, (X_test_high.shape[0], X_test_high.shape[1], 1))
predicted_stock_price_high = model_high.predict(X_test_high)
predicted_stock_price_high = sc.inverse_transform(predicted_stock_price_high)
predicted_stock_price_high

3/5 [=================>............] - ETA: 0s

C:\Users\91914\AppData\Local\Temp\ipykernel_22552\1960483048.py:2: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.

c:\Users\91914\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



5/5 [==============================] - 0s 30ms/step


array([[45393.664],
       [63477.15 ],
       [61218.23 ],
       [61252.875],
       [61762.   ],
       [61685.3  ],
       [62165.27 ],
       [60730.36 ],
       [58821.41 ],
       [58791.766],
       [56492.316],
       [57428.23 ],
       [57610.53 ],
       [56970.996],
       [55947.43 ],
       [55847.926],
       [57079.043],
       [56987.273],
       [53888.223],
       [55494.68 ],
       [56737.42 ],
       [56843.125],
       [56992.8  ],
       [55713.54 ],
       [55691.668],
       [52571.44 ],
       [48383.367],
       [48911.4  ],
       [49766.043],
       [48995.348],
       [48748.527],
       [48168.098],
       [47603.086],
       [48855.16 ],
       [48619.645],
       [46929.69 ],
       [47797.062],
       [47881.754],
       [46442.76 ],
       [45735.83 ],
       [46439.418],
       [45851.637],
       [47549.26 ],
       [48047.332],
       [49666.05 ],
       [50284.637],
       [49744.395],
       [49762.47 ],
       [50469.223],
       [49341.195],


### Low Value Prediction

In [34]:
dataset_total_low = pd.concat((df_data['Low'], dataset_test_low['Low']), axis = 0) # horizontal concatenation 1 vertical is 0 -> cols
inputs_low = dataset_total_low[len(dataset_total_low) - len(dataset_test_low) - 60:].values
inputs_low = inputs_low.reshape(-1,1)
inputs_low = sc.transform(inputs_low)
X_test_low = []

for i in range(60, len(inputs_low)): # for 6 months
    X_test_low.append(inputs_low[i-60:i, 0])
X_test_low = np.array(X_test_low)
X_test_low = np.reshape(X_test_low, (X_test_low.shape[0], X_test_low.shape[1], 1))
predicted_stock_price_low = model_low.predict(X_test_low)
predicted_stock_price_low = sc.inverse_transform(predicted_stock_price_low)
predicted_stock_price_low

C:\Users\91914\AppData\Local\Temp\ipykernel_22552\1619146979.py:2: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.

c:\Users\91914\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



5/5 [==============================] - 3s 38ms/step


array([[45242.957],
       [60984.754],
       [61869.785],
       [61515.312],
       [62540.723],
       [62336.957],
       [62477.195],
       [59475.965],
       [58982.785],
       [57036.17 ],
       [55981.906],
       [57338.746],
       [58228.703],
       [55982.047],
       [55995.344],
       [56055.02 ],
       [57054.902],
       [54059.19 ],
       [54024.902],
       [53669.53 ],
       [56520.715],
       [56092.613],
       [56718.53 ],
       [56189.31 ],
       [53065.504],
       [44632.008],
       [47430.69 ],
       [46713.023],
       [49392.406],
       [48621.07 ],
       [47371.66 ],
       [46997.938],
       [46839.055],
       [48526.62 ],
       [46201.14 ],
       [46504.613],
       [46661.08 ],
       [47401.27 ],
       [45802.76 ],
       [45631.586],
       [46402.13 ],
       [45580.805],
       [46616.78 ],
       [48318.508],
       [48139.758],
       [50377.316],
       [50298.664],
       [49855.207],
       [50606.14 ],
       [47922.746],


### Close Price Prediction

In [35]:
dataset_total_close = pd.concat((df_data['Close'], dataset_test_close['Close']), axis = 0) # horizontal concatenation 1 vertical is 0 -> cols
inputs_close = dataset_total_close[len(dataset_total_close) - len(dataset_test_close) - 60:].values
inputs_close = inputs_close.reshape(-1,1)
inputs_close = sc.transform(inputs_close)
X_test_close = []

for i in range(60, len(inputs_close)): # for 6 months
    X_test_close.append(inputs_close[i-60:i, 0])
X_test_close = np.array(X_test_close)
X_test_close = np.reshape(X_test_close, (X_test_close.shape[0], X_test_close.shape[1], 1))
predicted_stock_price_close = model_low.predict(X_test_close)
predicted_stock_price_close = sc.inverse_transform(predicted_stock_price_close)
predicted_stock_price_close

1/5 [=====>........................] - ETA: 0s

C:\Users\91914\AppData\Local\Temp\ipykernel_22552\3012050092.py:2: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.

c:\Users\91914\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



5/5 [==============================] - 0s 32ms/step


array([[46647.367],
       [62180.855],
       [62344.95 ],
       [62797.848],
       [63310.227],
       [63516.89 ],
       [62574.52 ],
       [60422.75 ],
       [60385.69 ],
       [57443.688],
       [58186.953],
       [59233.152],
       [58502.355],
       [56758.832],
       [57645.168],
       [56401.82 ],
       [57323.273],
       [54087.555],
       [55020.57 ],
       [57019.78 ],
       [57575.906],
       [57222.426],
       [57436.875],
       [56720.812],
       [54172.617],
       [50146.508],
       [49515.82 ],
       [50152.63 ],
       [50303.527],
       [50313.33 ],
       [47911.312],
       [47274.64 ],
       [49049.535],
       [49857.004],
       [47154.91 ],
       [46821.223],
       [48677.473],
       [47689.984],
       [46448.402],
       [46890.617],
       [46672.316],
       [46844.043],
       [48782.22 ],
       [48676.164],
       [50652.758],
       [50849.215],
       [50579.47 ],
       [50947.293],
       [50766.656],
       [48153.355],


### Combining Predicted Values with Date 

In [36]:
predicited_dates = df_data.iloc[imp_end_date_idx:, 0].values
df_pred = pd.DataFrame()
df_pred['Date'] = predicited_dates
df_pred['Open'] = np.array(predicted_stock_price_open)
df_pred['High'] = np.array(predicted_stock_price_high)
df_pred['Low'] = np.array(predicted_stock_price_low)
df_pred['Close'] = np.array(predicted_stock_price_close)
df_pred

,Date,Open,High,Low,Close
0,2021-11-10,44965.476562,45393.664062,45242.957031,46647.367188
1,2021-11-11,62427.851562,63477.148438,60984.753906,62180.855469
2,2021-11-12,60904.710938,61218.230469,61869.785156,62344.949219
3,2021-11-13,60886.953125,61252.875000,61515.312500,62797.847656
4,2021-11-14,61237.902344,61762.000000,62540.722656,63310.226562
...,...,...,...,...,...
141,2022-03-31,46077.058594,46328.011719,46965.097656,47331.890625
142,2022-04-01,45808.828125,46188.445312,45716.968750,45848.386719
143,2022-04-02,44437.347656,45362.621094,44545.871094,46338.117188
144,2022-04-03,44869.113281,45592.316406,45654.023438,45932.078125


In [37]:
plotting = go.Ohlc(x=df_data.iloc[imp_end_date_idx:, 0],
                   open=df_data.iloc[imp_end_date_idx:, 1],
                   high=df_data.iloc[imp_end_date_idx:, 2],
                   low=df_data.iloc[imp_end_date_idx:, 3],
                   close=df_data.iloc[imp_end_date_idx:, 4])

layout = dict(title="Stocks Plotting")
fig = go.Figure(data=[plotting], layout=layout)
fig.update(layout_xaxis_rangeslider_visible=False)

fig.show()

In [38]:
data_open = go.Scatter(x=df_pred['Date'],
                           y=df_pred['Open'],
                           name="Open Value")
data_high = go.Scatter(x=df_pred['Date'],
                           y=df_pred['High'],
                           name="High Value")
data_low = go.Scatter(x=df_pred['Date'],
                           y=df_pred['Low'],
                           name="Low Value")
data_close = go.Scatter(x=df_pred['Date'],
                           y=df_pred['Close'],
                           name="Close Value")


layout = dict(title="Predicted Stocks Plotting")
fig = go.Figure(data=[data_open, data_high, data_low, data_close], layout=layout)
fig.update(layout_xaxis_rangeslider_visible=False)

fig.show()

In [39]:
# Corrective Pattern points
wa_pred = WaveAnalyzer(df=df_pred, verbose=False)
wavepatterns_down_pred = []
wave_options_correction_pred = WaveOptionsGenerator3(up_to=9)
idx_start_cor = np.argmax(np.array(list(df_pred['High'])))
try:
    wave_cycles = set()
    for new_option_correction_pred in wave_options_correction_pred.options_sorted:
        waves_cor_pred = wa_pred.find_corrective_wave(idx_start=idx_start_cor, wave_config=new_option_correction_pred.values)

        if waves_cor_pred:
            wavepattern_cor_pred = WavePattern(waves_cor_pred, verbose=True)

            for rule in rules_to_check:

                if wavepattern_cor_pred.check_rule(rule):
                    if wavepattern_cor_pred in wavepatterns_down_pred:
                        continue
                    else:
                        wavepatterns_down_pred.append(wavepattern_cor_pred)
                        print(f'{rule.name} found: {new_option_correction_pred.values}')
    
except Exception as e:
    print("\n\n\n", e)
plot_pattern_two(df=df_pred, wave_pattern=wavepattern_cor_pred, title="Corrective Wave")
# print(df_pred)
wavepattern_cor_pred

Rule Violation of impulse for condition w2_1: End of Wave2 is lower than Start of Wave1.
Rule Violation of correction for condition w2_2: End of WaveB is higher than Start of WaveA.
Rule Violation of impulse for condition w2_1: End of Wave2 is lower than Start of Wave1.
Rule Violation of correction for condition w2_2: End of WaveB is higher than Start of WaveA.
Rule Violation of impulse for condition w2_1: End of Wave2 is lower than Start of Wave1.
Rule Violation of correction for condition w3_2: WaveB shorter (value wise) than 0.35 x WaveA.
Rule Violation of impulse for condition w2_1: End of Wave2 is lower than Start of Wave1.
Rule Violation of correction for condition w2_2: End of WaveB is higher than Start of WaveA.
Rule Violation of impulse for condition w2_1: End of Wave2 is lower than Start of Wave1.
Rule Violation of correction for condition w2_2: End of WaveB is higher than Start of WaveA.
Rule Violation of impulse for condition w2_1: End of Wave2 is lower than Start of Wave1.